In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [3]:
from we import get_we, initiate_model

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from scipy.stats import f_oneway

In [6]:
from sklearn.metrics import mean_absolute_error, accuracy_score

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [9]:
labels = [m['label'] for m in models]

In [10]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_unique_pos_we.csv'
                    )

In [11]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,Number,Gender,Lemma,POS,Tense,Person
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.909603,0.551025,0.436937,0.789782,0.311517,0.417476,0.629341,0.913722,0.349834,0.838988,...,0.285154,0.057887,0.030874,0.310204,invariable,feminine,2D,NOUN,NaN,NaN
3D,0.901779,0.549880,0.427826,0.797102,0.313900,0.419226,0.623478,0.916539,0.355807,0.844714,...,0.287610,0.054251,0.018604,0.305226,invariable,feminine,3D,NOUN,NaN,NaN
aa,0.899570,0.555891,0.418186,0.805754,0.319232,0.412976,0.618457,0.914183,0.367882,0.845985,...,0.287883,0.057476,0.013614,0.290463,invariable,masculine,aa,NOUN,NaN,NaN
abandon,0.898879,0.556306,0.417821,0.805256,0.318463,0.412505,0.618276,0.914983,0.367654,0.846577,...,0.288183,0.057827,0.014050,0.290994,singular,masculine,abandon,NOUN,NaN,NaN
abbaye,0.895888,0.554461,0.420659,0.799339,0.322441,0.418141,0.618820,0.921033,0.367005,0.845904,...,0.293985,0.057312,0.012175,0.298473,singular,feminine,abbaye,NOUN,NaN,NaN


In [12]:
# There are 6 extra feautres in addition to embedding dimensions in the file: number, gender, lemma, pos, tense, person
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'Number'

# Adjectivess

In [13]:
pos = ['ADJ', 'NOUN']

Split each model into train and test:

In [14]:
X_na_train = []
y_na_train = []

X_na_test = []
y_na_test = []

In [15]:
for we in we_with_features:
    xtr, xtst, ytr, ytst = prepare_dataset(dataset=we[(we.Gender != 'invariable') & \
                                                      (we.Number != 'invariable') & (we.POS.isin(pos))],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='plural',
                                           split=True,
                                           balance=True)
    X_na_train.append(xtr)
    X_na_test.append(xtst)
    
    y_na_train.append(ytr)
    y_na_test.append(ytst)

In [16]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['Train size'] = [len(x) for x in X_na_train]
dataset_sizes['Test size'] = [len(x) for x in X_na_test]
dataset_sizes

,Train size,Test size
flau_small_c,5212,1304
flau_base_u,6211,1553
flau_base_c,5212,1304
flau_large_c,5212,1304
cam_base,2148,538
xlm_large,422,106
xlm_base,422,106
bert_base_u,1553,389
distilbert_base,1027,257
bert_base_c,1027,257


## Non-independent dims

ANOVA test with p-value < 0.01:

In [17]:
anova_dims = [get_anova_dims(X_na_train[i], y_na_train[i]) for i in range(len(models))]

In [18]:
mi_dims = [get_mi_dims(X_na_train[i], y_na_train[i]) for i in range(len(models))]

In [19]:
non_ind_dims = [list(set(anova_dims[i]).intersection(mi_dims[i])) for i in range(len(models))]

Stats about the number of dimensions for each model:

In [20]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [21]:
non_ind_df['ANOVA'] = [len(x) for x in anova_dims]
non_ind_df['MI'] = [len(x) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x) for x in non_ind_dims]

In [22]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,383,366,301
flau_base_u,342,473,231
flau_base_c,505,526,375
flau_large_c,847,770,681
cam_base,13,367,10
xlm_large,175,596,122
xlm_base,9,447,6
bert_base_u,577,651,498
distilbert_base,270,502,205
bert_base_c,2,477,1


In [23]:
dims = {}

In [24]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = X_na_train[i].columns
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [25]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [26]:
lr_res = [lr(X_na_train[i], y_na_train[i]) for i in range(len(models))]

Train Perceptron 10 times and get average weights:

In [27]:
perceptron_res = [perceptron(X_na_train[i], y_na_train[i]) for i in range(len(models))]

Compute correlation to the gender vector:

In [28]:
corr_res = [correlation(X_na_train[i], y_na_train[i]) for i in range(len(models))]

In [29]:
for i in range(len(models)):
    for alpha in alphas:
        num_imp_dims = len(X_na_test[i].columns)*alpha//100
        lr_dims = [str(x[0]) for x in lr_res[i][:num_imp_dims]]
        perc_dims = [str(x[0]) for x in perceptron_res[i][:num_imp_dims]]
        corr_dims = [str(x[0]) for x in corr_res[i][:num_imp_dims]]
        dims[labels[i]][f'LR{alpha}'] = lr_dims
        dims[labels[i]][f'Perc{alpha}'] = perc_dims
        dims[labels[i]][f'Corr{alpha}'] = corr_dims
        dims[labels[i]][f'All imp dims{alpha}'] = list(set(lr_dims).intersection(perc_dims).intersection(corr_dims))

## Compute medians

In [30]:
medians = {}

In [31]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = {}
        dim_list = dims[labels[i]][dim_group]
        # Median of dimensions where feature vector is equal to 0
        median_0 = X_na_train[i][y_na_train[i] == 0][dim_list].median()
        # Median of dimensions where feature vector is equal to 1
        median_1 = X_na_train[i][y_na_train[i] == 1][dim_list].median()
        
        medians[labels[i]][dim_group]['0'] = median_0
        medians[labels[i]][dim_group]['1'] = median_1


In [32]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group])

In [33]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,512,768,768,1024,768,1024,768,768,768,768
ANOVA,383,342,505,847,13,175,9,577,270,2
MI,366,473,526,770,367,596,447,651,502,477
All non ind,301,231,375,681,10,122,6,498,205,1
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,2,2,1,3,0,0,2,0,0,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [34]:
y_preds = {}

In [35]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        dim_list = dims[labels[i]][dim_group]
        mae0 = X_na_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['0'], x), axis=1)
        mae1 = X_na_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['1'], x), axis=1)
        # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
        y_preds[labels[i]][dim_group] = (mae0 > mae1).apply(int)
    

## Compute accuracies

In [36]:
accs = {}

In [37]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_true = y_na_test[i]
        y_pred = y_preds[labels[i]][dim_group]
        if any(y_pred):
            acc = accuracy_score(y_true, y_pred)
        else:
            acc = 0
        accs[labels[i]][dim_group] = acc

In [38]:
accs_df = pd.DataFrame(accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.881902,0.602704,0.588957,0.941718,0.505576,0.603774,0.528302,0.542416,0.626459,0.575875
ANOVA,0.884969,0.609788,0.590491,0.942485,0.544610,0.603774,0.566038,0.542416,0.634241,0.587549
MI,0.888804,0.599485,0.602761,0.944785,0.514870,0.603774,0.528302,0.542416,0.645914,0.575875
All non ind,0.891871,0.602061,0.601994,0.944018,0.520446,0.613208,0.556604,0.544987,0.645914,0.591440
LR1,0.758436,0.550547,0.594325,0.865798,0.477695,0.518868,0.509434,0.542416,0.634241,0.564202
Perc1,0.743865,0.536381,0.686350,0.890337,0.520446,0.537736,0.547170,0.537275,0.607004,0.521401
Corr1,0.739264,0.531230,0.676380,0.903374,0.490706,0.622642,0.556604,0.539846,0.653696,0.595331
All imp dims1,0.715491,0.518352,0.658742,0.818252,0.000000,0.000000,0.481132,0.000000,0.000000,0.000000
LR5,0.851227,0.578880,0.632669,0.934816,0.498141,0.603774,0.556604,0.537275,0.614786,0.618677
Perc5,0.866564,0.595621,0.705521,0.939417,0.494424,0.584906,0.537736,0.537275,0.653696,0.579767


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [39]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANOVA,0.003067,0.007083,0.001534,0.000767,0.039033,0.000000,0.037736,0.000000,0.007782,0.011673
MI,0.006902,-0.003220,0.013804,0.003067,0.009294,0.000000,0.000000,0.000000,0.019455,0.000000
All non ind,0.009969,-0.000644,0.013037,0.002301,0.014870,0.009434,0.028302,0.002571,0.019455,0.015564
LR1,-0.123466,-0.052157,0.005368,-0.075920,-0.027881,-0.084906,-0.018868,0.000000,0.007782,-0.011673
Perc1,-0.138037,-0.066323,0.097393,-0.051380,0.014870,-0.066038,0.018868,-0.005141,-0.019455,-0.054475
Corr1,-0.142638,-0.071475,0.087423,-0.038344,-0.014870,0.018868,0.028302,-0.002571,0.027237,0.019455
All imp dims1,-0.166411,-0.084353,0.069785,-0.123466,-0.505576,-0.603774,-0.047170,-0.542416,-0.626459,-0.575875
LR5,-0.030675,-0.023825,0.043712,-0.006902,-0.007435,0.000000,0.028302,-0.005141,-0.011673,0.042802
Perc5,-0.015337,-0.007083,0.116564,-0.002301,-0.011152,-0.018868,0.009434,-0.005141,0.027237,0.003891


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [40]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c LR25 0.04907975460122693
flau_base_u Perc50 0.018673535093367666
flau_base_c Perc5 0.1165644171779141
flau_large_c LR25 0.017638036809815905
cam_base ANOVA 0.0390334572490707
xlm_large Corr1 0.018867924528301883
xlm_base ANOVA 0.037735849056603765
bert_base_u All non ind 0.0025706940874035134
distilbert_base LR10 0.05836575875486383
bert_base_c LR25 0.07003891050583666


In [41]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c LR25 0.9309815950920245
flau_base_u Perc50 0.6213779781068899
flau_base_c Perc5 0.7055214723926381
flau_large_c LR25 0.9593558282208589
cam_base ANOVA 0.5446096654275093
xlm_large Corr1 0.6226415094339622
xlm_base ANOVA 0.5660377358490566
bert_base_u All non ind 0.5449871465295629
distilbert_base LR10 0.6848249027237354
bert_base_c LR25 0.6459143968871596


In [42]:
if 'number_na.pickle' not in os.listdir('../Data/best_results/'):
    best_res = {}
    for label in labels:
        best_res[label] = {}
        best_res[label]['best_dim_set'] = []
        best_res[label]['best_dims'] = []
        best_res[label]['accs'] = []
        best_res[label]['gains'] = []
        best_res[label]['medians_0'] = []
        best_res[label]['medians_1'] = []


else:
    with open('../Data/best_results/number_na.pickle', 'rb') as f:
        best_res = pickle.load(f)

In [43]:
for i in range(len(models)):
    best_res[labels[i]]['best_dim_set'].append(gains_df.idxmax()[i])
    best_res[labels[i]]['best_dims'].append(dims[labels[i]][gains_df.idxmax()[i]])
    best_res[labels[i]]['accs'].append(accs_df.loc[accs_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['gains'].append(gains_df.loc[gains_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['medians_0'].append(medians[labels[i]][accs_df.idxmax()[i]]['0'])
    best_res[labels[i]]['medians_1'].append(medians[labels[i]][accs_df.idxmax()[i]]['1'])

In [44]:
with open('../Data/best_results/number_na.pickle', 'wb') as f:
    pickle.dump(best_res, f)

# Conclusion

1. FlauBERT small and large show very high accuracy for gender encoding in 25% of dimensions, found by the logistic regression, similar to other experiments.